In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
single = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\single_merged.csv')

In [3]:
single = single.set_index('timestamp')

### quickly convert it to a multiindex df

In [4]:
def convert_to_multi(df, timestamp):
    # creating most inner column 
    cols_str = ""
    cols = list(df.columns)
    for elem in cols:
        cols_str = "".join([cols_str, elem]) + " "
    len(cols_str.split())

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchanges_str = ""
    for col in cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchanges_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    features_str = "volume " * int(len(cols_str.split())/2) + "vwap " * int(len(cols_str.split())/2)

    new_df = pd.DataFrame(np.array(df), columns=[features_str.split(), exchanges_str.split(), cols_str.split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
    return new_df

In [5]:
df = convert_to_multi(single, pd.DataFrame(single.index))

In [6]:
df.head()

volume                                        \
                    binance_futures                                         
                           APE_USDT AVAX_USDT AXS_USDT BAKE_USDT BNB_USDT   
timestamp                                                                   
2019-09-08 17:57:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:58:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 17:59:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:00:00             NaN       NaN      NaN       NaN      NaN   
2019-09-08 18:01:00             NaN       NaN      NaN       NaN      NaN   

                                                                   ...  \
                                                                   ...   
                    BTC_BUSD BTC_USDT CRV_USDT CTK_USDT DOGE_USDT  ...   
timestamp                                                          ...   
2019-09-08 17:57:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 17:58:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 17:59:00      NaN    0.001      NaN      NaN       NaN  ...   
2019-09-08 18:00:00      NaN    0.000      NaN      NaN       NaN  ...   
2019-09-08 18:01:00      NaN    0.000      NaN      NaN       NaN  ...   

                        vwap                                          \
                         okx                                           
                    FTM_USDT GMT_USDT LINK_USDT MATIC_USDT NEAR_USDT   
timestamp                                                              
2019-09-08 17:57:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:58:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 17:59:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:00:00      NaN      NaN       NaN        NaN       NaN   
2019-09-08 18:01:00      NaN      NaN       NaN        NaN       NaN   

                                                                       
                                                                       
                    SAND_USDT SOL_USDT STORJ_USDT WAVES_USDT XRP_USDT  
timestamp                                                              
2019-09-08 17:57:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:58:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 17:59:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:00:00       NaN      NaN        NaN        NaN      NaN  
2019-09-08 18:01:00       NaN      NaN        NaN        NaN      NaN  

[5 rows x 180 columns]

### the above is the correct multiindex df

# next task: instantaneous arbitrage approximation
we will first assume no transaction costs
idea:

- for each coin, at any time (we can resample later), calculate maximum price dislocation across any exchange
    - buy lower one and sell higher one at the lower corresponding volume
- sum them all at the end

In [7]:
coins = list(df.columns.levels[2])
exchanges = list(df.columns.levels[1])
exchanges

['binance_futures', 'binance_spot', 'binanceus', 'okx']

# note that we can easily swap levels of the multiindex columns
df2 has the following
- outer column: the security (27 coins on different exchanges)
- middle column: the feature (volume or vwap)
- inner column: the exchange (binance_futures, binance_spots, binanceus, okx)

In [8]:
temp = df.swaplevel(0, 2, 1).sort_index(axis=1)
df2 = temp.swaplevel(1, 2, 1).sort_index(axis=1)
df2

APE_USDT                             \
                             volume                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN       154.52       0.0 NaN   
2023-01-31 23:56:00             NaN       264.84       0.0 NaN   
2023-01-31 23:57:00             NaN       287.58       0.0 NaN   
2023-01-31 23:58:00             NaN       887.80       0.0 NaN   
2023-01-31 23:59:00             NaN       404.81       0.0 NaN   

                                                                \
                               vwap                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:56:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:57:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:58:00             NaN     9.411818   9.78506 NaN   
2023-01-31 23:59:00             NaN     9.411817   9.78506 NaN   

                          AVAX_USDT               ...   WAVES_USDT      \
                             volume               ...         vwap       
                    binance_futures binance_spot  ... binance_spot okx   
timestamp                                         ...                    
2019-09-08 17:57:00             NaN          NaN  ...          NaN NaN   
2019-09-08 17:58:00             NaN          NaN  ...          NaN NaN   
2019-09-08 17:59:00             NaN          NaN  ...          NaN NaN   
2019-09-08 18:00:00             NaN          NaN  ...          NaN NaN   
2019-09-08 18:01:00             NaN          NaN  ...          NaN NaN   
...                             ...          ...  ...          ...  ..   
2023-01-31 23:55:00             NaN       599.82  ...    10.713475 NaN   
2023-01-31 23:56:00             NaN       584.41  ...    10.713472 NaN   
2023-01-31 23:57:00             NaN      1333.10  ...    10.713472 NaN   
2023-01-31 23:58:00             NaN       642.07  ...    10.713444 NaN   
2023-01-31 23:59:00             NaN       304.73  ...    10.713440 NaN   

                           XRP_USDT                             \
                             volume                              
                    binance_futures binance_spot binanceus okx   
timestamp                                                        
2019-09-08 17:57:00             NaN          NaN       NaN NaN   
2019-09-08 17:58:00             NaN          NaN       NaN NaN   
2019-09-08 17:59:00             NaN          NaN       NaN NaN   
2019-09-08 18:00:00             NaN          NaN       NaN NaN   
2019-09-08 18:01:00             NaN          NaN       NaN NaN   
...                             ...          ...       ...  ..   
2023-01-31 23:55:00             NaN     135505.0   11914.6 NaN   
2023-01-31 23:56:00             NaN     177663.0   11914.6 NaN   
2023-01-31 23:57:00             NaN     197800.0   11914.6 NaN   
2023-01-31 23:58:00             NaN     159875.0   11914.6 NaN   
2023-01-31

let's make this algorithm for 1 coin for now, say Bitcoin. let us denote $\mu =$ instance profit.

$$\mu = \min(volume_i, volume_j) * (price_j - price_i)$$

arbitrage index df

                  price_i        price_j          i                j              volume[i]     volume[j]      instance_profit 
        9:30        10              8         binanceus     binance_futures           2            5              
        9:31        12              8            okx        binance_futures           3            7

where $i, j$ are the same coin but on different exchanges. they represent different indices in the df.

Let coin $price_i$ be the minimum price of some arbitrary coin and let $price_j$ be the maximum price of the same arbirary coin at some time $t$

We can separate the min volume column into two columns volume_j, volume_i -> this works too

First, let us get all of our needed columns so we can achieve all calculations vectorially

In [84]:
btc = pd.DataFrame(df2.index)
btc = btc.set_index('timestamp')
btc['price_i'] = df2['BTC_USDT']['vwap'][exchanges].min(axis=1)
btc['price_j'] = df2['BTC_USDT']['vwap'][exchanges].max(axis=1)
btc['i'] = df2['BTC_USDT']['vwap'][exchanges].idxmin(axis=1)
btc['j'] = df2['BTC_USDT']['vwap'][exchanges].idxmax(axis=1)
btc['binance_futures'] = df2['BTC_USDT']['volume']['binance_futures']
btc['binance_spot'] = df2['BTC_USDT']['volume']['binance_spot']
btc['binanceus'] = df2['BTC_USDT']['volume']['binanceus']
btc['okx'] = df2['BTC_USDT']['volume']['okx']
btc

,price_i,price_j,i,j,binance_futures,binance_spot,binanceus,okx
timestamp,,,,,,,,
2019-09-08 17:57:00,10000.000000,10000.000000,binance_futures,binance_futures,0.001,NaN,NaN,NaN
2019-09-08 17:58:00,10000.000000,10000.000000,binance_futures,binance_futures,0.000,NaN,NaN,NaN
2019-09-08 17:59:00,10000.000000,10000.000000,binance_futures,binance_futures,0.001,NaN,NaN,NaN
2019-09-08 18:00:00,10000.000000,10000.000000,binance_futures,binance_futures,0.000,NaN,NaN,NaN
2019-09-08 18:01:00,10000.000000,10000.000000,binance_futures,binance_futures,0.000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,NaN,132.98878,0.665210,NaN
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,NaN,186.18800,2.498794,NaN
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,NaN,110.49868,1.791509,NaN


let's see how many of these columns are even useful (ie. $i \neq j \implies \ \exists$ a price dislocation)

In [85]:
btc2 = btc.loc[btc['i'] != btc['j']]
display(btc2)
print("Percentage of minutes where there exists some price dislocation for Bitcoin", len(btc2)/len(df2))

,price_i,price_j,i,j,binance_futures,binance_spot,binanceus,okx
timestamp,,,,,,,,
2021-01-01 00:00:00,11072.608906,28943.335000,binance_futures,binanceus,143.661,27.457032,0.138756,NaN
2021-01-01 00:01:00,11072.650364,28986.477367,binance_futures,binanceus,256.280,58.477501,1.543793,NaN
2021-01-01 00:02:00,11072.666971,28986.958822,binance_futures,binanceus,102.675,42.470329,0.094843,NaN
2021-01-01 00:03:00,11072.680837,28986.846904,binance_futures,binanceus,85.762,30.360677,0.042083,NaN
2021-01-01 00:04:00,11072.692196,28986.856765,binance_futures,binanceus,70.246,24.124339,0.019755,NaN
...,...,...,...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,NaN,132.988780,0.665210,NaN
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,NaN,186.188000,2.498794,NaN
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,NaN,110.498680,1.791509,NaN


Percentage of minutes where there exists some price dislocation for Bitcoin 0.6130909554761603


let us group the dfs by $i, j$ pairs such that we can perform only vectorial operations

In [86]:
grouped = btc2.groupby(['i', 'j'])
dfs = [grouped.get_group(x) for x in grouped.groups]
dfs[0]

,price_i,price_j,i,j,binance_futures,binance_spot,binanceus,okx
timestamp,,,,,,,,
2021-01-01 00:13:00,11073.001476,28888.692690,binance_futures,binance_spot,833.321,104.225054,4.481732,NaN
2021-01-01 00:14:00,11073.096066,28858.591061,binance_futures,binance_spot,593.694,156.587294,0.121057,NaN
2021-01-01 00:15:00,11073.135287,28852.909088,binance_futures,binance_spot,245.781,45.695385,0.131859,NaN
2021-01-01 00:16:00,11073.177407,28851.097380,binance_futures,binance_spot,263.391,28.328974,0.638687,NaN
2021-01-01 00:17:00,11073.208158,28849.253612,binance_futures,binance_spot,192.205,34.035514,0.336274,NaN
...,...,...,...,...,...,...,...,...
2022-11-29 14:25:00,28501.413711,30908.325982,binance_futures,binance_spot,329.912,286.404550,1.916272,NaN
2022-11-29 14:26:00,28501.409425,30908.296342,binance_futures,binance_spot,150.774,148.803240,0.640509,NaN
2022-11-29 14:27:00,28501.398588,30908.240427,binance_futures,binance_spot,380.892,280.566640,2.130832,NaN


In [87]:
for df in dfs:
    exchange1, exchange2 = df['i'].unique()[0], df['j'].unique()[0]
    df['min_volume'] = df[[exchange1, exchange2]].min(axis=1)
dfs[1] # example df grouped by i, j

,price_i,price_j,i,j,binance_futures,binance_spot,binanceus,okx,min_volume
timestamp,,,,,,,,,
2021-01-01 00:00:00,11072.608906,28943.335000,binance_futures,binanceus,143.661,27.457032,0.138756,NaN,0.138756
2021-01-01 00:01:00,11072.650364,28986.477367,binance_futures,binanceus,256.280,58.477501,1.543793,NaN,1.543793
2021-01-01 00:02:00,11072.666971,28986.958822,binance_futures,binanceus,102.675,42.470329,0.094843,NaN,0.094843
2021-01-01 00:03:00,11072.680837,28986.846904,binance_futures,binanceus,85.762,30.360677,0.042083,NaN,0.042083
2021-01-01 00:04:00,11072.692196,28986.856765,binance_futures,binanceus,70.246,24.124339,0.019755,NaN,0.019755
...,...,...,...,...,...,...,...,...,...
2021-09-29 08:55:00,27064.881594,42274.588369,binance_futures,binanceus,65.152,NaN,0.000000,NaN,0.000000
2021-09-29 08:56:00,27064.884571,42274.588369,binance_futures,binanceus,44.958,NaN,0.000000,NaN,0.000000
2021-09-29 08:57:00,27064.888731,42274.588369,binance_futures,binanceus,62.860,NaN,0.000000,NaN,0.000000


In [88]:
final_btc = pd.concat(dfs)
final_btc = final_btc.sort_index()
final_btc = final_btc.drop(columns=['binance_futures', 'binance_spot', 'binanceus', 'okx'])
final_btc

,price_i,price_j,i,j,min_volume
timestamp,,,,,
2021-01-01 00:00:00,11072.608906,28943.335000,binance_futures,binanceus,0.138756
2021-01-01 00:01:00,11072.650364,28986.477367,binance_futures,binanceus,1.543793
2021-01-01 00:02:00,11072.666971,28986.958822,binance_futures,binanceus,0.094843
2021-01-01 00:03:00,11072.680837,28986.846904,binance_futures,binanceus,0.042083
2021-01-01 00:04:00,11072.692196,28986.856765,binance_futures,binanceus,0.019755
...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,0.665210
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,2.498794
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,1.791509


finally we have obtained a complete df our desired format, and so now we can easily recompute the instant profits as the following:
$$\mu = \min(volume_i, volume_j) * (price_j - price_i)$$

In [89]:
final_btc['instant_profit'] = final_btc['min_volume'] * abs(final_btc['price_i'] - final_btc['price_j'])
final_btc

,price_i,price_j,i,j,min_volume,arbitrage_index
timestamp,,,,,,
2021-01-01 00:00:00,11072.608906,28943.335000,binance_futures,binanceus,0.138756,2479.670470
2021-01-01 00:01:00,11072.650364,28986.477367,binance_futures,binanceus,1.543793,27655.240731
2021-01-01 00:02:00,11072.666971,28986.958822,binance_futures,binanceus,0.094843,1699.045182
2021-01-01 00:03:00,11072.680837,28986.846904,binance_futures,binanceus,0.042083,753.881851
2021-01-01 00:04:00,11072.692196,28986.856765,binance_futures,binanceus,0.019755,353.894321
...,...,...,...,...,...,...
2023-01-31 23:55:00,27207.485001,28969.897150,binanceus,binance_spot,0.665210,1172.374186
2023-01-31 23:56:00,27207.471198,28969.884672,binanceus,binance_spot,2.498794,4403.908214
2023-01-31 23:57:00,27207.461297,28969.877264,binanceus,binance_spot,1.791509,3157.384066


In [90]:
final_btc['instance_profit'].sum()

738046599255.8605

# now let's extend this to all the coins

should we represent this as potentially a hashmap of dataframes, or a multiindex

- i think hashmap for now

In [110]:
def calculate_immediate_arb(df):
    
    # get all necessary columns for vectorial operations
    coin = pd.DataFrame(df.index)
    coin = coin.set_index('timestamp')
    coin['price_i'] = df['vwap'][exchanges].min(axis=1)
    coin['price_j'] = df['vwap'][exchanges].max(axis=1)
    coin['i'] = df['vwap'][exchanges].idxmin(axis=1)
    coin['j'] = df['vwap'][exchanges].idxmax(axis=1)
    coin['binance_futures'] = df['volume']['binance_futures']
    coin['binance_spot'] = df['volume']['binance_spot']
    coin['binanceus'] = df['volume']['binanceus']
    coin['okx'] = df['volume']['okx']

    # only consider rows where there exists a price dislocation and i != j != NaN
    coin2 = coin.loc[coin['i'] != coin['j']]
    coin2 = coin2.dropna(subset=['i','j'])

    # group by i, j pairs
    grouped = coin2.groupby(['i', 'j'])
    dfs = [grouped.get_group(x) for x in grouped.groups]
    
    # compute the min_volume column for each grouped df
    for x in dfs:
        exchange1, exchange2 = x['i'].unique()[0], x['j'].unique()[0]
        x['min_volume'] = x[[exchange1, exchange2]].min(axis=1)
    
    # concatenate them, sort by tiemstamp, and drop unnecessary columns
    final_coin = pd.concat(dfs)
    final_coin = final_coin.sort_index()
    final_coin = final_coin.drop(columns=['binance_futures', 'binance_spot', 'binanceus', 'okx'])
    final_coin['instant_profit'] = final_coin['min_volume'] * abs(final_coin['price_i'] - final_coin['price_j'])
    return final_coin

In [114]:
immediate_arb = {}
for coin in coins:
    print(coin)
    immediate_arb[coin] = calculate_immediate_arb(df2[coin])

APE_USDT
AVAX_USDT
AXS_USDT
BAKE_USDT


KeyError: "['binanceus', 'okx'] not in index"

In [116]:
df2['BAKE_USDT'] # note how this coin is only traded on binace_futures and binance_spot, so i can't hardcode...

volume                         vwap             
                    binance_futures binance_spot binance_futures binance_spot
timestamp                                                                    
2019-09-08 17:57:00             NaN          NaN             NaN          NaN
2019-09-08 17:58:00             NaN          NaN             NaN          NaN
2019-09-08 17:59:00             NaN          NaN             NaN          NaN
2019-09-08 18:00:00             NaN          NaN             NaN          NaN
2019-09-08 18:01:00             NaN          NaN             NaN          NaN
...                             ...          ...             ...          ...
2023-01-31 23:55:00             NaN        255.1             NaN     1.478914
2023-01-31 23:56:00             NaN        816.3             NaN     1.478914
2023-01-31 23:57:00             NaN          0.0             NaN     1.478914
2023-01-31 23:58:00             NaN       6072.7             NaN     1.478914
2023-01-31 23:59:00             NaN       1184.4             NaN     1.478914

[1787402 rows x 4 columns]